In [1]:
from pyspark.ml.feature import *
from pyspark.sql.session import SparkSession
from pyspark.ml import *
from pyspark.sql import *
from pyspark.sql.functions import *
from functools import *

In [2]:
# spark session 필수 설정
spark = SparkSession.builder.appName("Test").config("spark.executor.instances", "1").config("spark.executor.cores", "1").config("spark.executor.memory", "2g").config("spark.driver.memory", "2g").getOrCreate()


In [3]:
from pyspark.ml.linalg import *
import numpy as np
import scipy.sparse as sps


In [10]:

denseVec = Vectors.dense(1.0,2.0,3.0)
size=3
idx=[1,2]
values=[2.0,3.0]
sparseVec = Vectors.sparse(size, idx, values)
sparseVec
# denseVec


SparseVector(3, {1: 2.0, 2: 3.0})

In [13]:
# dense 
v1 = np.array([0.1, 0.0, 0.2, 0.3])
v2 = Vectors.dense([0.1, 0.0, 0.2, 0.3])
v3 = [0.1, 0.0, 0.2, 0.3]


In [78]:
# sparse 
v3 = Vectors.sparse(4, [(0, 0.1), (2, 0.2), (3, 0.3)])
v4 = Vectors.sparse(4, [0, 2, 3], [0.1, 0.2, 0.3])
v5 = sps.csc_matrix(np.array([0.1, 0.2, 0.3,1,2,3]),(np.array([0,0,1,1,2,2]), np.array([0,1,0,1,1,0,1])), [shape = (3,2)])
print()
print(v2)
print()
print(v3.toArray())
print()
print(v4.toArray())
print()
print(v5)
display(v5)

SyntaxError: invalid syntax (<ipython-input-78-53544328a00c>, line 4)

In [42]:
v5.todense()
print(v5.toarray())

[[0.1]
 [0. ]
 [0.2]
 [0.3]]


In [32]:
a = Vectors.sparse(4, {1: 1.0, 3: 5.5})
print(a.toArray())

[0.  1.  0.  5.5]


In [4]:
df = spark.read.json("/home/lab01/data/exam/simple-ml")

In [5]:
df.orderBy("value2").show()

+-----+----+------+------------------+
|color| lab|value1|            value2|
+-----+----+------+------------------+
|green|good|     1|14.386294994851129|
|green| bad|    16|14.386294994851129|
| blue| bad|     8|14.386294994851129|
| blue| bad|     8|14.386294994851129|
| blue| bad|    12|14.386294994851129|
|green| bad|    16|14.386294994851129|
|green|good|    12|14.386294994851129|
|  red|good|    35|14.386294994851129|
|  red|good|    35|14.386294994851129|
|  red| bad|     2|14.386294994851129|
|  red| bad|    16|14.386294994851129|
|  red| bad|    16|14.386294994851129|
| blue| bad|     8|14.386294994851129|
|green|good|     1|14.386294994851129|
|green|good|    12|14.386294994851129|
| blue| bad|     8|14.386294994851129|
|  red|good|    35|14.386294994851129|
| blue| bad|    12|14.386294994851129|
|  red| bad|    16|14.386294994851129|
|green|good|    12|14.386294994851129|
+-----+----+------+------------------+
only showing top 20 rows



In [13]:
supervised = RFormula(formula="lab ~. +color:value1 + color:value2")

In [16]:
fittedRF = supervised.fit(df)
preparedDF = fittedRF.transform(df)
preparedDF.show(20, False)

+-----+----+------+------------------+----------------------------------------------------------------------+-----+
|color|lab |value1|value2            |features                                                              |label|
+-----+----+------+------------------+----------------------------------------------------------------------+-----+
|green|good|1     |14.386294994851129|(10,[1,2,3,5,8],[1.0,1.0,14.386294994851129,1.0,14.386294994851129])  |1.0  |
|blue |bad |8     |14.386294994851129|(10,[2,3,6,9],[8.0,14.386294994851129,8.0,14.386294994851129])        |0.0  |
|blue |bad |12    |14.386294994851129|(10,[2,3,6,9],[12.0,14.386294994851129,12.0,14.386294994851129])      |0.0  |
|green|good|15    |38.97187133755819 |(10,[1,2,3,5,8],[1.0,15.0,38.97187133755819,15.0,38.97187133755819])  |1.0  |
|green|good|12    |14.386294994851129|(10,[1,2,3,5,8],[1.0,12.0,14.386294994851129,12.0,14.386294994851129])|1.0  |
|green|bad |16    |14.386294994851129|(10,[1,2,3,5,8],[1.0,16.0,14.38629

In [17]:
supervised2 = RFormula(formula="lab ~. ")

In [18]:
fittedRF2 = supervised2.fit(df)
preparedDF2 = fittedRF2.transform(df)
preparedDF2.show(20, False)

+-----+----+------+------------------+---------------------------------+-----+
|color|lab |value1|value2            |features                         |label|
+-----+----+------+------------------+---------------------------------+-----+
|green|good|1     |14.386294994851129|[0.0,1.0,1.0,14.386294994851129] |1.0  |
|blue |bad |8     |14.386294994851129|[0.0,0.0,8.0,14.386294994851129] |0.0  |
|blue |bad |12    |14.386294994851129|[0.0,0.0,12.0,14.386294994851129]|0.0  |
|green|good|15    |38.97187133755819 |[0.0,1.0,15.0,38.97187133755819] |1.0  |
|green|good|12    |14.386294994851129|[0.0,1.0,12.0,14.386294994851129]|1.0  |
|green|bad |16    |14.386294994851129|[0.0,1.0,16.0,14.386294994851129]|0.0  |
|red  |good|35    |14.386294994851129|[1.0,0.0,35.0,14.386294994851129]|1.0  |
|red  |bad |1     |38.97187133755819 |[1.0,0.0,1.0,38.97187133755819]  |0.0  |
|red  |bad |2     |14.386294994851129|[1.0,0.0,2.0,14.386294994851129] |0.0  |
|red  |bad |16    |14.386294994851129|[1.0,0.0,16.0,

In [20]:
train, test = preparedDF.randomSplit([0.7,0.3])

In [23]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(labelCol="label", featuresCol="features")


In [25]:
print(lr.explainParams())

aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0)
family: The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial (default: auto)
featuresCol: features column name. (default: features, current: features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label, current: label)
lowerBoundsOnCoefficients: The lower bounds on coefficients if fitting under bound constrained optimization. The bound matrix must be compatible with the shape (1, number of features) for binomial regression, or (number of classes, number of features) for multinomial regression. (undefined)
lowerBoundsOnIntercepts: The lower bounds on intercepts if fitting under bound constrained optimization. The

In [26]:
fittedLR = lr.fit(train)

In [27]:
fittedLR.transform(train).select("label","prediction").show()

+-----+----------+
|label|prediction|
+-----+----------+
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
+-----+----------+
only showing top 20 rows



In [55]:
train, test = df.randomSplit([0.7,0.3])

rForm = RFormula()
lr = LogisticRegression().setLabelCol("label").setFeaturesCol("features")

In [56]:
stages = [rForm, lr]
pipeline = Pipeline().setStages(stages)

In [57]:
from pyspark.ml.tuning import ParamGridBuilder

params = ParamGridBuilder().addGrid(rForm.formula,
                                   ["lab ~ .+ color:value1",
                                   "lab ~ .+ color:value1 + color:value2"])\
.addGrid(lr.elasticNetParam, [0.0,0.5,1.0])\
.addGrid(lr.regParam, [0.1,2.0]).build()

In [58]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator().setMetricName("areaUnderROC").setRawPredictionCol("prediction")\
.setLabelCol("label")



In [59]:
from pyspark.ml.tuning import TrainValidationSplit

tvs = TrainValidationSplit()\
.setTrainRatio(0.75)\
.setEstimatorParamMaps(params)\
.setEstimator(pipeline)\
.setEvaluator(evaluator)

In [60]:
tvsFitted = tvs.fit(train)

In [61]:
evaluator.evaluate(tvsFitted.transform(test))

0.90625

In [62]:
scaleDF = spark.read.parquet("/home/lab01/data/exam/simple-ml-scaling/")

In [64]:
sScaler = StandardScaler().setInputCol("features")
sScaler.fit(scaleDF).transform(scaleDF).show(5,False)


+---+--------------+------------------------------------------------------------+
|id |features      |StandardScaler_78e7b9d1fa73__output                         |
+---+--------------+------------------------------------------------------------+
|0  |[1.0,0.1,-1.0]|[1.1952286093343936,0.02337622911060922,-0.5976143046671968]|
|1  |[2.0,1.1,1.0] |[2.390457218668787,0.2571385202167014,0.5976143046671968]   |
|0  |[1.0,0.1,-1.0]|[1.1952286093343936,0.02337622911060922,-0.5976143046671968]|
|1  |[2.0,1.1,1.0] |[2.390457218668787,0.2571385202167014,0.5976143046671968]   |
|1  |[3.0,10.1,3.0]|[3.5856858280031805,2.3609991401715313,1.7928429140015902]  |
+---+--------------+------------------------------------------------------------+



In [88]:
conDF = spark.range(20).selectExpr("cast(id as double)")

In [89]:
conDF.show()

+----+
|  id|
+----+
| 0.0|
| 1.0|
| 2.0|
| 3.0|
| 4.0|
| 5.0|
| 6.0|
| 7.0|
| 8.0|
| 9.0|
|10.0|
|11.0|
|12.0|
|13.0|
|14.0|
|15.0|
|16.0|
|17.0|
|18.0|
|19.0|
+----+



In [70]:
from pyspark.ml.feature import Bucketizer

In [90]:
bucketBorders = [-1.0 , 0.5, 5.0, 10.0, 15.0, 20.0 ]
bucketer = Bucketizer().setSplits(bucketBorders).setInputCol("id")
bucketer.transform(conDF).show()


+----+-------------------------------+
|  id|Bucketizer_60b17a1e6889__output|
+----+-------------------------------+
| 0.0|                            0.0|
| 1.0|                            1.0|
| 2.0|                            1.0|
| 3.0|                            1.0|
| 4.0|                            1.0|
| 5.0|                            2.0|
| 6.0|                            2.0|
| 7.0|                            2.0|
| 8.0|                            2.0|
| 9.0|                            2.0|
|10.0|                            3.0|
|11.0|                            3.0|
|12.0|                            3.0|
|13.0|                            3.0|
|14.0|                            3.0|
|15.0|                            4.0|
|16.0|                            4.0|
|17.0|                            4.0|
|18.0|                            4.0|
|19.0|                            4.0|
+----+-------------------------------+



In [91]:
minMax = MinMaxScaler().setMin(0).setMax(1).setInputCol("id")
fittedminMax = minMax.fit(conDF)
fittedminMax.transform(scaleDF).show()

IllegalArgumentException: 'requirement failed: Column id must be of type struct<type:tinyint,size:int,indices:array<int>,values:array<double>> but was actually double.'

In [94]:
manhattanDistance = Normalizer().setP(1).setInputCol("features")
manhattanDistance.transform(scaleDF).show(5,False)

+---+--------------+---------------------------------------------------------------+
|id |features      |Normalizer_34568f82d63c__output                                |
+---+--------------+---------------------------------------------------------------+
|0  |[1.0,0.1,-1.0]|[0.47619047619047616,0.047619047619047616,-0.47619047619047616]|
|1  |[2.0,1.1,1.0] |[0.48780487804878053,0.26829268292682934,0.24390243902439027]  |
|0  |[1.0,0.1,-1.0]|[0.47619047619047616,0.047619047619047616,-0.47619047619047616]|
|1  |[2.0,1.1,1.0] |[0.48780487804878053,0.26829268292682934,0.24390243902439027]  |
|1  |[3.0,10.1,3.0]|[0.18633540372670807,0.6273291925465838,0.18633540372670807]   |
+---+--------------+---------------------------------------------------------------+



In [93]:
lblIndxr = StringIndexer().setInputCol("lab").setOutputCol("labelInd")
idxRes = lblIndxr.fit(df).transform(df)
idxRes.show()

+-----+----+------+------------------+--------+
|color| lab|value1|            value2|labelInd|
+-----+----+------+------------------+--------+
|green|good|     1|14.386294994851129|     1.0|
| blue| bad|     8|14.386294994851129|     0.0|
| blue| bad|    12|14.386294994851129|     0.0|
|green|good|    15| 38.97187133755819|     1.0|
|green|good|    12|14.386294994851129|     1.0|
|green| bad|    16|14.386294994851129|     0.0|
|  red|good|    35|14.386294994851129|     1.0|
|  red| bad|     1| 38.97187133755819|     0.0|
|  red| bad|     2|14.386294994851129|     0.0|
|  red| bad|    16|14.386294994851129|     0.0|
|  red|good|    45| 38.97187133755819|     1.0|
|green|good|     1|14.386294994851129|     1.0|
| blue| bad|     8|14.386294994851129|     0.0|
| blue| bad|    12|14.386294994851129|     0.0|
|green|good|    15| 38.97187133755819|     1.0|
|green|good|    12|14.386294994851129|     1.0|
|green| bad|    16|14.386294994851129|     0.0|
|  red|good|    35|14.386294994851129|  

In [96]:
oneIndxr = StringIndexer().setInputCol("color").setOutputCol("colorInd")
colorLab = oneIndxr.fit(df).transform(df.select("color")) 
ohe = OneHotEncoder().setInputCol("colorInd")
ohe.transform(colorLab).show()

+-----+--------+----------------------------------+
|color|colorInd|OneHotEncoder_3606108bafb7__output|
+-----+--------+----------------------------------+
|green|     1.0|                     (2,[1],[1.0])|
| blue|     2.0|                         (2,[],[])|
| blue|     2.0|                         (2,[],[])|
|green|     1.0|                     (2,[1],[1.0])|
|green|     1.0|                     (2,[1],[1.0])|
|green|     1.0|                     (2,[1],[1.0])|
|  red|     0.0|                     (2,[0],[1.0])|
|  red|     0.0|                     (2,[0],[1.0])|
|  red|     0.0|                     (2,[0],[1.0])|
|  red|     0.0|                     (2,[0],[1.0])|
|  red|     0.0|                     (2,[0],[1.0])|
|green|     1.0|                     (2,[1],[1.0])|
| blue|     2.0|                         (2,[],[])|
| blue|     2.0|                         (2,[],[])|
|green|     1.0|                     (2,[1],[1.0])|
|green|     1.0|                     (2,[1],[1.0])|
|green|     

In [99]:
sales = spark.read.format("csv").option("header","true").option("inferSchema","true")\
.load("/home/lab01/data/exam/retail-data/by-day/*.csv")\
.coalesce(5).where("Description IS NOT NULL")

In [100]:
# 그냥 토크나이징
tkn = Tokenizer().setInputCol("Description").setOutputCol("DescOut")
tokenized = tkn.transform(sales.select("Description"))
tokenized.show(10,False)

+-------------------------------+-------------------------------------+
|Description                    |DescOut                              |
+-------------------------------+-------------------------------------+
|RABBIT NIGHT LIGHT             |[rabbit, night, light]               |
|DOUGHNUT LIP GLOSS             |[doughnut, lip, gloss]               |
|12 MESSAGE CARDS WITH ENVELOPES|[12, message, cards, with, envelopes]|
|BLUE HARMONICA IN BOX          |[blue, harmonica, in, box]           |
|GUMBALL COAT RACK              |[gumball, coat, rack]                |
|SKULLS  WATER TRANSFER TATTOOS |[skulls, , water, transfer, tattoos] |
|FELTCRAFT GIRL AMELIE KIT      |[feltcraft, girl, amelie, kit]       |
|CAMOUFLAGE LED TORCH           |[camouflage, led, torch]             |
|WHITE SKULL HOT WATER BOTTLE   |[white, skull, hot, water, bottle]   |
|ENGLISH ROSE HOT WATER BOTTLE  |[english, rose, hot, water, bottle]  |
+-------------------------------+-------------------------------

In [101]:
# 정규표현식 사용 토크나이징
rt = RegexTokenizer()\
.setInputCol("Description")\
.setOutputCol("DescOut")\
.setPattern(" ")\
.setToLowercase(True)
rt.transform(sales.select("Description")).show(10,False)

+-------------------------------+-------------------------------------+
|Description                    |DescOut                              |
+-------------------------------+-------------------------------------+
|RABBIT NIGHT LIGHT             |[rabbit, night, light]               |
|DOUGHNUT LIP GLOSS             |[doughnut, lip, gloss]               |
|12 MESSAGE CARDS WITH ENVELOPES|[12, message, cards, with, envelopes]|
|BLUE HARMONICA IN BOX          |[blue, harmonica, in, box]           |
|GUMBALL COAT RACK              |[gumball, coat, rack]                |
|SKULLS  WATER TRANSFER TATTOOS |[skulls, water, transfer, tattoos]   |
|FELTCRAFT GIRL AMELIE KIT      |[feltcraft, girl, amelie, kit]       |
|CAMOUFLAGE LED TORCH           |[camouflage, led, torch]             |
|WHITE SKULL HOT WATER BOTTLE   |[white, skull, hot, water, bottle]   |
|ENGLISH ROSE HOT WATER BOTTLE  |[english, rose, hot, water, bottle]  |
+-------------------------------+-------------------------------

In [102]:
# 값 패턴 포착
rt = RegexTokenizer()\
.setInputCol("Description")\
.setOutputCol("DescOut")\
.setPattern(" ")\
.setGaps(False)\
.setToLowercase(True)
rt.transform(sales.select("Description")).show(10,False)

+-------------------------------+---------------+
|Description                    |DescOut        |
+-------------------------------+---------------+
|RABBIT NIGHT LIGHT             |[ ,  ]         |
|DOUGHNUT LIP GLOSS             |[ ,  ,  ]      |
|12 MESSAGE CARDS WITH ENVELOPES|[ ,  ,  ,  ]   |
|BLUE HARMONICA IN BOX          |[ ,  ,  ,  ]   |
|GUMBALL COAT RACK              |[ ,  ]         |
|SKULLS  WATER TRANSFER TATTOOS |[ ,  ,  ,  ,  ]|
|FELTCRAFT GIRL AMELIE KIT      |[ ,  ,  ]      |
|CAMOUFLAGE LED TORCH           |[ ,  ]         |
|WHITE SKULL HOT WATER BOTTLE   |[ ,  ,  ,  ,  ]|
|ENGLISH ROSE HOT WATER BOTTLE  |[ ,  ,  ,  ]   |
+-------------------------------+---------------+
only showing top 10 rows



In [105]:
# 단어 제거

englishStopWords = StopWordsRemover.loadDefaultStopWords("english")
stops = StopWordsRemover()\
.setStopWords(englishStopWords)\
.setInputCol("DescOut")

stops.transform(tokenized).show(10, False)

+-------------------------------+-------------------------------------+-------------------------------------+
|Description                    |DescOut                              |StopWordsRemover_7f822c0f1938__output|
+-------------------------------+-------------------------------------+-------------------------------------+
|RABBIT NIGHT LIGHT             |[rabbit, night, light]               |[rabbit, night, light]               |
|DOUGHNUT LIP GLOSS             |[doughnut, lip, gloss]               |[doughnut, lip, gloss]               |
|12 MESSAGE CARDS WITH ENVELOPES|[12, message, cards, with, envelopes]|[12, message, cards, envelopes]      |
|BLUE HARMONICA IN BOX          |[blue, harmonica, in, box]           |[blue, harmonica, box]               |
|GUMBALL COAT RACK              |[gumball, coat, rack]                |[gumball, coat, rack]                |
|SKULLS  WATER TRANSFER TATTOOS |[skulls, , water, transfer, tattoos] |[skulls, , water, transfer, tattoos] |
|FELTCRAFT

In [106]:
unigram = NGram().setInputCol("DescOut").setN(1)
bigram = NGram().setInputCol("DescOut").setN(2)
trigram = NGram().setInputCol("DescOut").setN(3)

unigram.transform(tokenized.select("DescOut")).show(10,False)

bigram.transform(tokenized.select("DescOut")).show(10,False)

trigram.transform(tokenized.select("DescOut")).show(10,False)

+-------------------------------------+-------------------------------------+
|DescOut                              |NGram_2bc3a3467cbb__output           |
+-------------------------------------+-------------------------------------+
|[rabbit, night, light]               |[rabbit, night, light]               |
|[doughnut, lip, gloss]               |[doughnut, lip, gloss]               |
|[12, message, cards, with, envelopes]|[12, message, cards, with, envelopes]|
|[blue, harmonica, in, box]           |[blue, harmonica, in, box]           |
|[gumball, coat, rack]                |[gumball, coat, rack]                |
|[skulls, , water, transfer, tattoos] |[skulls, , water, transfer, tattoos] |
|[feltcraft, girl, amelie, kit]       |[feltcraft, girl, amelie, kit]       |
|[camouflage, led, torch]             |[camouflage, led, torch]             |
|[white, skull, hot, water, bottle]   |[white, skull, hot, water, bottle]   |
|[english, rose, hot, water, bottle]  |[english, rose, hot, wate

In [107]:
# 단어를 숫자로 변환하기

cv = CountVectorizer()\
.setInputCol("DescOut")\
.setOutputCol("countVec")\
.setVocabSize(500)\
.setMinTF(1)\
.setMinDF(2)

fittedCV = cv.fit(tokenized)
fittedCV.transform(tokenized).show(10,False)

+-------------------------------+-------------------------------------+---------------------------------------------+
|Description                    |DescOut                              |countVec                                     |
+-------------------------------+-------------------------------------+---------------------------------------------+
|RABBIT NIGHT LIGHT             |[rabbit, night, light]               |(500,[150,185,212],[1.0,1.0,1.0])            |
|DOUGHNUT LIP GLOSS             |[doughnut, lip, gloss]               |(500,[462,463,492],[1.0,1.0,1.0])            |
|12 MESSAGE CARDS WITH ENVELOPES|[12, message, cards, with, envelopes]|(500,[35,41,166],[1.0,1.0,1.0])              |
|BLUE HARMONICA IN BOX          |[blue, harmonica, in, box]           |(500,[10,16,36,352],[1.0,1.0,1.0,1.0])       |
|GUMBALL COAT RACK              |[gumball, coat, rack]                |(500,[228,280,407],[1.0,1.0,1.0])            |
|SKULLS  WATER TRANSFER TATTOOS |[skulls, , water, trans

In [109]:
tfIdfIn = tokenized\
.where("array_contains(DescOut, 'red')")\
.select("DescOut")\
.limit(10)
tfIdfIn.show(10,False)

+---------------------------------------+
|DescOut                                |
+---------------------------------------+
|[gingham, heart, , doorstop, red]      |
|[red, floral, feltcraft, shoulder, bag]|
|[alarm, clock, bakelike, red]          |
|[pin, cushion, babushka, red]          |
|[red, retrospot, mini, cases]          |
|[red, kitchen, scales]                 |
|[gingham, heart, , doorstop, red]      |
|[large, red, babushka, notebook]       |
|[red, retrospot, oven, glove]          |
|[red, retrospot, plate]                |
+---------------------------------------+



In [110]:
tf = HashingTF()\
.setInputCol("DescOut")\
.setOutputCol("TFOut")\
.setNumFeatures(10000)

idf = IDF()\
.setInputCol("TFOut")\
.setOutputCol("IDFOut")\
.setMinDocFreq(2)

idf.fit(tf.transform(tfIdfIn)).transform(tf.transform(tfIdfIn)).show(10,False)

+---------------------------------------+--------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------+
|DescOut                                |TFOut                                                   |IDFOut                                                                                                              |
+---------------------------------------+--------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------+
|[gingham, heart, , doorstop, red]      |(10000,[3372,4291,4370,6594,9160],[1.0,1.0,1.0,1.0,1.0])|(10000,[3372,4291,4370,6594,9160],[1.2992829841302609,0.0,1.2992829841302609,1.2992829841302609,1.2992829841302609])|
|[red, floral, feltcraft, shoulder, bag]|(10000,[155,1152,4291,5981,6756],[1.0,1.0,1.0,1.0,1.0]) |(10000,[155,1152,4291,5981,6756],[0.0,